In [169]:
import sys

import mysql.connector as mc
mydb = mc.connect(
    host      = "yangtze.csc.ncsu.edu",
    user      = "appreview",
    passwd    = "mas!321",
    database  = "app_reviews"
)
mycursor = mydb.cursor()
mycursor.execute("SHOW DATABASEs")
myresult = mycursor.fetchall()
for x in myresult:
    print(x)

('app_reviews',)
('app_reviews_larger',)
('information_schema',)


In [170]:
print('Fetching reviews ...')

NUM_REVIEWS_EACH_SET = 5000

# Facebook App
appID = 284882215

# total reviews: 949,210
# 1 star reviews: 361,109
# 5 star reviews: 241,783

positive_reviews = []
negative_reviews = []

mycursor.execute('select title, body, date from reviews where appId = {} and rating = 5 limit {}'.format(appID, 
                                                                                                         NUM_REVIEWS_EACH_SET))
myresult = mycursor.fetchall()
for x in myresult:
    positive_reviews.append(x)
print('# negative reviews:', len(positive_reviews))

mycursor.execute('select title, body, date from reviews where appId = {} and rating = 1 limit {}'.format(appID, 
                                                                                                         NUM_REVIEWS_EACH_SET))
myresult = mycursor.fetchall()
for x in myresult:
    negative_reviews.append(x)
print('# negative reviews:', len(negative_reviews))

mydb.close()

Fetching reviews ...
# negative reviews: 5000
# negative reviews: 5000


In [166]:
import pandas as pd
import numpy as np

def get_news_data(data, media):
    
    filter1 = data['publication'] == media
    news = data.loc[filter1]
    index = np.random.choice(len(news), 5000, replace=False)
    news = news.sample(n = 5000) 
    news = [(row['title'],row['content'], row['publication']) for index, row in news.iterrows()]
    
    return news

data = pd.read_csv('../News_Media_bias/Git Code/Dataset/articles1.csv')
print(set(data['publication']))
#sys.exit()
left_news = get_news_data(data, 'New York Times')
right_news = get_news_data(data, 'CNN')

print(len(left_news), len(right_news))

{'Breitbart', 'Business Insider', 'Atlantic', 'New York Times', 'CNN'}
5000 5000


In [171]:
import spacy

sys.stdout.write('Loading SpaCy model...\n')
lg = spacy.load('en_core_web_lg')
sys.stdout.write('Loaded.\n')

Loading SpaCy model...
Loaded.


In [172]:
# consider only adjectives

# first, convert each review into a bag of adjectives

positive_bags = []
sys.stdout.write("Processing positive reviews...\n")
for ridx, tp in enumerate(positive_reviews):     #change left_news to positive_reviews to run for reviews
    if (ridx + 1) % 1000 ==0:
        sys.stdout.write('Processing ... {}/{}\n'.format(ridx+1, len(positive_reviews)))
        
    bag = []     
    text = tp[1].encode('utf-8', 'ignore').decode('utf-8', 'ignore')
    tokens = lg(text, disable=['parser', 'ner'])
    for token in tokens:
        if token.tag_ == 'JJ':
            bag.append(token.lemma_)
    positive_bags.append(set(bag))
sys.stdout.write('Done. \n')


negative_bags = []
sys.stdout.write("Processing negative reviews...\n")
for ridx, tp in enumerate(negative_reviews):     #change right_news to negative_reviews to run for reviews
    if (ridx + 1) % 1000 ==0:
        sys.stdout.write('Processing ... {}/{}\n'.format(ridx+1, len(negative_reviews)))
        
    bag = []     
    text = tp[1].encode('utf-8', 'ignore').decode('utf-8', 'ignore')
    tokens = lg(text, disable=['parser', 'ner'])
    for token in tokens:
        if token.tag_ == 'JJ':
            bag.append(token.lemma_)
    negative_bags.append(set(bag))
sys.stdout.write('Done. \n')

Processing positive reviews...
Processing ... 1000/5000
Processing ... 2000/5000
Processing ... 3000/5000
Processing ... 4000/5000
Processing ... 5000/5000
Done. 
Processing negative reviews...
Processing ... 1000/5000
Processing ... 2000/5000
Processing ... 3000/5000
Processing ... 4000/5000
Processing ... 5000/5000
Done. 


In [173]:
print(type(negative_bags), negative_bags[0])

<class 'list'> {'used', 'constant', 'hard', 'awful'}


In [193]:
MAX_FREQUENCY = 500 # out of 10000
MIN_FREQUENCY = 2 # out of 10000

In [198]:
def get_pickle_data(filepath):
    
    import pickle
    ent_clusters = None
    with open(filepath, 'rb') as f:
        ent_clusters = pickle.load(f)
    return ent_clusters

def get_candidate_adj_counts(ent_adj_media_1, ent_adj_media_2, entity, MAX_FREQUENCY, MIN_FREQUENCY):
    
    positive_counts = ent_adj_media_1[entity]['adjectives']
    negative_counts = ent_adj_media_2[entity]['adjectives']
    
    combined = positive_counts.copy()
    for word in negative_counts:
        if word in combined:
            combined[word] = combined[word] + negative_counts[word]
        else:
            combined[word] = negative_counts[word]

    # keep words based on frequency
    kept = {}
    for word in combined:
        if combined[word] >= MIN_FREQUENCY and combined[word] <= MAX_FREQUENCY:
            kept[word] = combined[word]
    sys.stdout.write('Kept words: {}\n'.format(len(kept)))

    return kept
    
    
media_1_file = "Nammed_entities/ent_adj_pairs_Breitbart.pickle"
media_2_file = "Nammed_entities/ent_adj_pairs_NYT.pickle"
entity_adj_pairs_media_1 = get_pickle_data(media_1_file)
entity_adj_pairs_media_2 = get_pickle_data(media_2_file)

entities1 = list(entity_adj_pairs_media_1.keys())
entities2 = list(entity_adj_pairs_media_2.keys())
combined_entities = list(set(entities1) & set(entities2)) 

print("Number of entities in Media 1 ", len(entities1))
print("Number of entities in Media 2 ", len(entities2))
print("Number of entities in both ", len(combined_entities))
    
print(len(entity_adj_pairs_media_1))
print(len(entity_adj_pairs_media_2))

for entity in combined_entities:
    kept = get_candidate_adj_counts(entity_adj_pairs_media_1, entity_adj_pairs_media_2, entity, MAX_FREQUENCY, MIN_FREQUENCY)
    if(len(kept) > 25):
        break

print(len(kept))

Number of entities in Media 1  1015
Number of entities in Media 2  1393
Number of entities in both  693
1015
1393
Kept words: 0
Kept words: 59
59


In [181]:
# Second, count frequencies

sys.stdout.write('Counting...\n')
positive_counts = {}
for bag in positive_bags:
    for word in bag:
        if word in positive_counts:
            positive_counts[word] = positive_counts[word] + 1
        else:
            positive_counts[word] = 1

negative_counts = {}
for bag in negative_bags:
    for word in bag:
        if word in negative_counts:
            negative_counts[word] = negative_counts[word] + 1
        else:
            negative_counts[word] = 1

print(list((x, negative_counts[x]) for x in negative_counts)[0:5])

Counting...
[('disgusting', 5), ('unethical', 1), ('pothead', 1), ('resent', 1), ('hard', 51)]
Kept words: 183


In [ ]:
# Total counts
combined = positive_counts.copy()
for word in negative_counts:
    if word in combined:
        combined[word] = combined[word] + negative_counts[word]
    else:
        combined[word] = negative_counts[word]

# keep words based on frequency
kept = {}
for word in combined:
    if combined[word] >= MIN_FREQUENCY and combined[word] <= MAX_FREQUENCY:
        kept[word] = combined[word]
sys.stdout.write('Kept words: {}\n'.format(len(kept)))

In [203]:
print(kept)

{'former': 12, 'hard': 2, 'populist': 45, 'clear': 3, 'willing': 2, 'Turkish': 3, 'high': 2, 'unsafe': 2, 'French': 6, 'able': 4, 'prime': 3, 'nationalist': 7, 'strong': 2, 'inflammatory': 3, 'critical': 2, 'much': 2, 'last': 3, 'major': 2, 'American': 3, 'popular': 3, 'same': 2, 'public': 3, 'Moroccan': 10, 'political': 12, 'next': 2, 'general': 5, 'long': 2, 'likely': 5, 'positive': 2, 'ongoing': 3, 'such': 4, 'many': 2, 'own': 2, 'guilty': 6, 'recent': 9, 'free': 3, 'holy': 2, 'Muslim': 2, 'foreign': 2, 'European': 4, 'important': 2, 'right': 3, 'North': 3, 'liberal': 5, 'Dutch': 124, 'incumbent': 3, 'junior': 2, 'new': 6, 'centrist': 4, 'gay': 2, 'British': 2, 'Eurosceptic': 16, 'Islamic': 4, 'far': 2, 'national': 5, 'presidential': 3, 'other': 6, 'first': 3, 'African': 3}


In [201]:
# Third, get synonyms to get word pairs

from nltk.corpus import wordnet as wn
# use Lesk: http://www.nltk.org/howto/wsd.html

word_pairs = []
for word in kept:
    for syn in wn.synsets(word):
        for l in syn.lemmas():
            if l.name() == word:
                continue
            if l.name() in kept:
                word_pairs.append(sorted([word, l.name()]))
                
word_pairs = [t for t in (set(tuple(wp) for wp in word_pairs))] 

sys.stdout.write('#Candidate word pairs: {}\n'.format(len(word_pairs)))

#Candidate word pairs: 4


In [202]:
def get_disparity(word_pairs):
    
    import math

    tool = []
    for word1, word2 in word_pairs:
        f_pos_1 = float(positive_counts[word1]) if word1 in positive_counts else 0.1
        f_pos_2 = float(positive_counts[word2]) if word2 in positive_counts else 0.1
        f_neg_1 = float(negative_counts[word1]) if word1 in negative_counts else 0.1
        f_neg_2 = float(negative_counts[word2]) if word2 in negative_counts else 0.1

        disparity1 = f_pos_1 / f_neg_1
        disparity2 = f_neg_2 / f_pos_2
        disparity = f_pos_1 * f_neg_2 / f_neg_1 / f_pos_2

        if disparity < 1.0:
            # word 1 is negative
            tool.append((word2, 1.0/disparity2, word1, 1.0/disparity1))
        else:
            tool.append((word1, disparity1, word2, disparity2))

    tool.sort(key=lambda x: x[1]*x[3], reverse=True)

    for tp in tool:
        if tp[1]>=1.0 and tp[3]>=1.0:
            print(tp)
            
get_disparity(word_pairs)

('strong', 1.0, 'hard', 5.1)
('free', 2.0, 'liberal', 1.0)
('Islamic', 1.0, 'Muslim', 1.0)


In [154]:
# Fourth, rank word pairs by frequency disparity
# disparity is calculated as: 
# -- count(pos, word1) * count(neg, word2) / count(neg, word1) / count(pos, word2)
# -- or its reciprocal, whichever is bigger

import math

tool = []
for word1, word2 in word_pairs:
    f_pos_1 = float(positive_counts[word1]) if word1 in positive_counts else 0.1
    f_pos_2 = float(positive_counts[word2]) if word2 in positive_counts else 0.1
    f_neg_1 = float(negative_counts[word1]) if word1 in negative_counts else 0.1
    f_neg_2 = float(negative_counts[word2]) if word2 in negative_counts else 0.1
    
    disparity1 = f_pos_1 / f_neg_1
    disparity2 = f_neg_2 / f_pos_2
    disparity = f_pos_1 * f_neg_2 / f_neg_1 / f_pos_2
    
    if disparity < 1.0:
        # word 1 is negative
        tool.append((word2, 1.0/disparity2, word1, 1.0/disparity1))
    else:
        tool.append((word1, disparity1, word2, disparity2))

tool.sort(key=lambda x: x[1]*x[3], reverse=True)

for tp in tool:
    if tp[1]>=1.0 and tp[3]>=1.0:
        print(tp)

('oversize', 270.0, 'outsized', 7.5)
('finished', 4.333333333333333, 'end', 230.0)
('close', 1.5534979423868314, 'end', 230.0)
('oversize', 270.0, 'oversized', 1.2)
('fifth', 1.1495327102803738, '5th', 200.0)
('outsize', 27.5, 'outsized', 7.5)
('fine', 2.1320754716981134, 'ok', 40.5)
('primitive', 5.0, 'naive', 17.0)
('outsize', 27.5, 'oversized', 1.2)
('sixth', 1.7083333333333335, '6th', 14.0)
('crisp', 10.5, 'frosty', 2.0)
('fine', 2.1320754716981134, 'okay', 8.5)
('acerbic', 9.0, 'vitriolic', 1.8571428571428572)
('caustic', 9.0, 'vitriolic', 1.8571428571428572)
('prodigious', 12.0, 'olympian', 1.3333333333333333)
('mute', 13.0, 'damp', 1.2)
('dreaded', 9.0, 'horrific', 1.3653846153846154)
('second', 1.3167539267015707, '2nd', 9.0)
('grievous', 11.0, 'grave', 1.0666666666666667)
('dreaded', 9.0, 'horrendous', 1.2727272727272727)
('coming', 2.3333333333333335, 'upcoming', 4.875)
('grievous', 11.0, 'heartbreaking', 1.0277777777777777)
('dreaded', 9.0, 'dreadful', 1.1666666666666667)
('

In [156]:
count = 0
for tp in tool:
    if tp[1]>=1.5 and tp[3]>=1.5:
        print(tp)
        count += 1
print(count)

('oversize', 270.0, 'outsized', 7.5)
('finished', 4.333333333333333, 'end', 230.0)
('close', 1.5534979423868314, 'end', 230.0)
('outsize', 27.5, 'outsized', 7.5)
('fine', 2.1320754716981134, 'ok', 40.5)
('primitive', 5.0, 'naive', 17.0)
('sixth', 1.7083333333333335, '6th', 14.0)
('crisp', 10.5, 'frosty', 2.0)
('fine', 2.1320754716981134, 'okay', 8.5)
('acerbic', 9.0, 'vitriolic', 1.8571428571428572)
('caustic', 9.0, 'vitriolic', 1.8571428571428572)
('coming', 2.3333333333333335, 'upcoming', 4.875)
('unseemly', 2.8, 'indecent', 3.4)
('revelatory', 4.0, 'suggestive', 2.25)
('unwarranted', 4.25, 'unjustified', 2.0)
('accustomed', 4.363636363636363, 'habitual', 1.8)
('forthcoming', 1.5925925925925926, 'upcoming', 4.875)
('primal', 4.5, 'key', 1.6970684039087947)
('worrisome', 3.5, 'distressing', 1.8571428571428572)
('teenage', 2.2608695652173916, 'teen', 2.6666666666666665)
('homeless', 2.4411764705882355, 'stateless', 2.3333333333333335)
('consummate', 3.6666666666666665, 'masterful', 1.5

In [160]:
l = []
for tp in tool:
    if(tp[1] > 1 and tp[3]>1):
        l.append((tp[0], tp[2]))
print(l)

[('oversize', 'outsized'), ('finished', 'end'), ('close', 'end'), ('oversize', 'oversized'), ('fifth', '5th'), ('outsize', 'outsized'), ('fine', 'ok'), ('primitive', 'naive'), ('outsize', 'oversized'), ('sixth', '6th'), ('crisp', 'frosty'), ('fine', 'okay'), ('acerbic', 'vitriolic'), ('caustic', 'vitriolic'), ('prodigious', 'olympian'), ('mute', 'damp'), ('dreaded', 'horrific'), ('second', '2nd'), ('grievous', 'grave'), ('dreaded', 'horrendous'), ('coming', 'upcoming'), ('grievous', 'heartbreaking'), ('dreaded', 'dreadful'), ('unseemly', 'indecent'), ('first', '1st'), ('mortal', 'deadly'), ('revelatory', 'suggestive'), ('unwarranted', 'unjustified'), ('accustomed', 'habitual'), ('forthcoming', 'upcoming'), ('primal', 'key'), ('ninth', '9th'), ('torn', 'tear'), ('improbable', 'unbelievable'), ('fearsome', 'horrific'), ('gloomy', 'sorry'), ('torn', 'snap'), ('worrisome', 'distressing'), ('perplexing', 'puzzling'), ('fearsome', 'horrendous'), ('dreary', 'sorry'), ('drab', 'sorry'), ('teen